In [1]:
!pip install transformers peft bitsandbytes trl deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 40.3 MB/s eta 0:00:00
     ━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig
from trl import SFTTrainer

In [4]:
# setup the quantization config

compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

In [2]:
# from google.colab import userdata
# userdata.get('######')

In [6]:
# Ensure local directory does not exist
!rm -rf meta-llama/Meta-Llama-3-8B


# Load base model with access token
base_model_name = 'meta-llama/Meta-Llama-3-8B'
access_token = "##################"
llama_3 = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    token=access_token,
    quantization_config=quant_config,
    device_map={"": 0},
)

# Load tokenizer with access token
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    token=access_token,
    trust_remote_code=True,
)

# Set padding token and side
tokenizer.pad_token = tokenizer.eos_token # this model doesnt requires separate padding token
tokenizer.padding_side = "right" #padding should be added to the end (right side) of the sequences

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Function to generate a personalized email automatically
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name, sender_name, company_name, website_url):
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Sender Name: {sender_name}\n"
        f"Company Name: {company_name}\n"
        f"Website URL: {website_url}\n"
        f"Email Content:\nSubject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=250,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content
    email_content_start = email.find("Subject:")  # Start of the generated content
    generated_email = email[email_content_start:].strip()

    # Ensure the email is approximately 150 words
    words = generated_email.split()
    if len(words) > 150:
        generated_email = " ".join(words[:150]) + "..."

    return generated_email

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 85,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju",
        "sender_name": "Binit Sapkota",
        "company_name": "MrBeerGod Technologies",
        "website_url": "https://barahsinghe.com/"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code = inputs["discount_code"],
        user_name=inputs["user_name"],
        sender_name=inputs["sender_name"],
        company_name=inputs["company_name"],
        website_url = inputs['website_url']
    )
    print(email_content)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

We're excited to offer you an exclusive discount on our top-selling Beer. As a valued customer, we'd like to extend a special discount of 20% off your purchase, using the code Jadija.

This offer is only available to our most loyal customers, so don't miss out! To take advantage of this limited-time deal, simply enter the code at checkout. You'll be able to enjoy our delicious Beer at an unbeatable price.

Thank you for being a loyal customer. We appreciate your business and look forward to serving you again soon.

Best regards,
Binit Sapota
MrBeerGod


In [10]:
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name, sender_name, company_name, website_url):
    # Determine email tone and persuasion based on lead score
    if lead_score > 80:
        tone = "excited and convincing"
    elif lead_score >= 50:
        tone = "enthusiastic"
    else:
        tone = "informative"

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Sender Name: {sender_name}\n"
        f"Company Name: {company_name}\n"
        f"Website URL: {website_url}\n"
        f"Email Tone: {tone}\n"
        f"Email Content:\nSubject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=250,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content
    email_content_start = email.find("Subject:")  # Start of the generated content
    generated_email = email[email_content_start:].strip()

    # Ensure the email is approximately 150 words
    words = generated_email.split()
    if len(words) > 150:
        generated_email = " ".join(words[:150]) + "..."

    return generated_email

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 85,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju",
        "sender_name": "Binit Sapkota",
        "company_name": "MrBeerGod Technologies",
        "website_url": "https://barahsinghe.com/"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code=inputs["discount_code"],
        user_name=inputs["user_name"],
        sender_name=inputs["sender_name"],
        company_name=inputs["company_name"],
        website_url=inputs["website_url"]
    )
    print(email_content)


Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

As a valued customer, we would like to extend a special offer to you on our Beer products. We are offering a 20% discount on all products, with a unique discount code Jadija. This offer is available for a limited time only, so don't miss out!

Our Beer products are known for their high quality and delicious taste. They are made from the finest ingredients and are perfect for any occasion. Whether you are hosting a party or just want to relax at home, our Beer is the perfect choice.

Don't miss this opportunity to save on your favorite Beer


In [25]:
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name, sender_name, company_name, website_url):
    # Determine email tone and persuasion based on lead score
    if lead_score > 80:
        tone = "excited and convincing"
    elif lead_score >= 50:
        tone = "enthusiastic"
    else:
        tone = "informative"

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Sender Name: {sender_name}\n"
        f"Company Name: {company_name}\n"
        f"Website URL: {website_url}\n"
        f"Email Tone: {tone}\n"
        f"Email Content:\nSubject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=500,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content
    email_content_start = email.find("Subject:")  # Start of the generated content
    generated_email = email[email_content_start:].strip()

    # Ensure the email is approximately 150 words
    words = generated_email.split()
    if len(words) > 500:
        generated_email = " ".join(words[:500]) + "..."

    return generated_email

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 95,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju",
        "sender_name": "Binit Sapkota",
        "company_name": "MrBeerGod Technologies",
        "website_url": "https://barahsinghe.com/"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code=inputs["discount_code"],
        user_name=inputs["user_name"],
        sender_name=inputs["sender_name"],
        company_name=inputs["company_name"],
        website_url=inputs["website_url"]
    )
    print(email_content)


Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

We've got a special offer just for you! Our team has been working hard to bring you the best deal on Beer. With a Lead Score of 95 and a 20% discount, this offer is not to be missed.

Discount Code:
Jadiya

Product Name:
Beer

Product Description:
Beer is a delicious alcoholic beverage that has been enjoyed for centuries. It is made from fermenting grains, such as barley, wheat, or rice, and adding hops for flavor. Beer comes in a variety of styles, from light, crisp lagers to dark, rich stouts. It's the perfect beverage for any occasion, whether you're celebrating a special event or just relaxing with friends.

Don't miss out on this exclusive offer! Click the link below to redeem your discount code and start enjoying the taste of Beer.

https://barasingshe.com/beer

Regards,
Binit Sapota
MrBeerGod


In [ ]:
### need to add some guards.

In [29]:
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name):
    # Determine email tone and persuasion based on lead score
    if lead_score > 80:
        tone = "excited and convincing"
    elif lead_score >= 50:
        tone = "enthusiastic"
    else:
        tone = "informative"

    # Fixed details
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Fixed Details:\n"
        f"Sender Name: {sender_name}\n"
        f"Company Name: {company_name}\n"
        f"Website URL: {website_url}\n"
        f"Email Content:\nSubject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=250,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content
    email_content_start = email.find("Subject:")  # Start of the generated content
    generated_email = email[email_content_start:].strip()

    # Ensure the email is approximately 150 words
    words = generated_email.split()
    if len(words) > 150:
        generated_email = " ".join(words[:150]) + "..."

    return generated_email

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 85,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code=inputs["discount_code"],
        user_name=inputs["user_name"]
    )
    print(email_content)


Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

I'm excited to offer you an exclusive discount on our best-selling Beer! As a loyal customer, you're eligible for a 20% discount on all products, and we've included a special discount code for your convenience. This is a limited-time offer, so don't miss out on the chance to save big!

Discount Code:
Jadiya

To take advantage of this offer, simply enter the discount code at checkout. We hope you enjoy our Beer as much as we do!

Best regards,
Binit Sapokota
MrBeerGod


In [34]:
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"

    # Determine email tone and persuasion based on lead score
    if lead_score > 80:
        tone = "excited and convincing"
    elif lead_score >= 50:
        tone = "enthusiastic"
    else:
        tone = "informative"

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Sender Name: {sender_name}\n"
        f"Company Name: {company_name}\n"
        f"Website URL: {website_url}\n"
        f"Email Tone: {tone}\n"
        f"Email Content:\nSubject: Exclusive {product_name} Offer Just for You!\n"
        # f"Dear {user_name},\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=250,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content
    email_content_start = email.find("Subject:")  # Start of the generated content
    generated_email = email[email_content_start:].strip()

    # Ensure the email is approximately 150 words
    words = generated_email.split()
    if len(words) > 150:
        generated_email = " ".join(words[:150]) + "..."

    # Email template with placeholders for generated content
    email_template = (
        f"Subject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
        f"{generated_email}\n\n"
        f"Best regards,\n"
        f"{sender_name}\n"
        f"{company_name}\n"
        f"Visit us: {website_url}\n"
    )

    return email_template

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 85,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code=inputs["discount_code"],
        user_name=inputs["user_name"]
    )
    print(email_content)



Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

Subject: Exclusive Beer Offer Just for You!
Hello Ms. Ms. Jadiyaa,
We hope you’re doing well. We’re reaching out to you today with an exclusive offer on our premium Beer products.
As a valued customer, we’d like to extend a special discount of 20% on all our Beer products. Use the discount code “Jadiya” to avail this offer. We believe this is a great opportunity for you to try out our products and experience the taste of Beer for yourself.
We’re confident that our Beer will meet your expectations and exceed your taste buds. Our products are carefully crafted to provide a unique and delicious taste

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/



In [36]:
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name):
    # Fixed template components
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"

    # Determine email tone and persuasion based on lead score
    if lead_score > 80:
        tone = "excited and convincing"
    elif lead_score >= 50:
        tone = "enthusiastic"
    else:
        tone = "informative"

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Email Content: Dear {user_name},\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=250,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email_body = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content and remove any additional subject or greeting lines
    email_content_start = email_body.find("Dear")  # Start of the generated content
    generated_email_body = email_body[email_content_start:].strip()

    # Ensure the email is approximately 150 words
    words = generated_email_body.split()
    if len(words) > 150:
        generated_email_body = " ".join(words[:150]) + "..."

    # Final email template
    final_email = (
        f"Subject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
        f"{generated_email_body}\n\n"
        f"Best regards,\n"
        f"{sender_name}\n"
        f"{company_name}\n"
        f"Visit us: {website_url}\n"
    )

    return final_email

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 85,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code=inputs["discount_code"],
        user_name=inputs["user_name"]
    )
    print(email_content)


Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

Dear Ms. Sriju,

As a customer, I understand that you value your time and money, and I appreciate your interest in our product, Beer. I'm writing to offer you a special discount on our product.

Our product, [Beer], is a high-quality, handcrafted beer made with the finest ingredients and brewed with care. We believe in providing our customers with the best possible experience, and our product reflects that commitment.

As a token of our appreciation, we are offering a 20% discount on all purchases made through our website. To take advantage of this offer, simply enter the code "[Jadiya]" at checkout.

Thank you for your interest and support. We look forward to serving you and providing you with the highest quality products and services.

Sincerely,
[Your

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/



In [38]:
def generate_email_auto(product_name, product_description, lead_score, discount_offers, discount_code, user_name):
    # Fixed template components
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"

    # Determine email tone and persuasion based on lead score
    if lead_score > 80:
        tone = "excited and convincing"
    elif lead_score >= 50:
        tone = "enthusiastic"
    else:
        tone = "informative"

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email with the following details:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Lead Score: {lead_score}\n"
        f"Discount Offers: {discount_offers}\n"
        f"Discount Code: {discount_code}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Email Content:\n"
        f"Dear {user_name},\n\n"
        f"Please write the email content below this line.\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=250,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email_body = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content and ensure there are no duplicate greetings
    email_content_start = email_body.find("Dear")  # Start of the generated content
    email_content = email_body[email_content_start:].strip()

    # Split by lines and remove duplicate greetings
    email_lines = email_content.split('\n')
    if email_lines[1].startswith("Dear"):
        email_lines = email_lines[1:]

    # Ensure the email is approximately 150 words
    generated_email_body = " ".join(email_lines).strip()
    words = generated_email_body.split()
    if len(words) > 150:
        generated_email_body = " ".join(words[:150]) + "..."

    # Final email template
    final_email = (
        f"Subject: Exclusive {product_name} Offer Just for You!\n"
        f"Dear {user_name},\n\n"
        f"{generated_email_body}\n\n"
        f"Best regards,\n"
        f"{sender_name}\n"
        f"{company_name}\n"
        f"Visit us: {website_url}\n"
    )

    return final_email

# Example inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "lead_score": 85,
        "discount_offers": "20%",
        "discount_code": "Jadiya",
        "user_name": "Ms. Sriju"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        lead_score=inputs["lead_score"],
        discount_offers=inputs["discount_offers"],
        discount_code=inputs["discount_code"],
        user_name=inputs["user_name"]
    )
    print(email_content)


Subject: Exclusive Beer Offer Just for You!
Dear Ms. Sriju,

Dear Ms. Sriju,  Please write the email content below this line. Regards, [Your Name] [Your Company] [Company Logo]  Thanks for your interest in our products. We understand that you are looking for a new supplier for your business, and we are confident that we can provide you with the best possible products and services.  Our products are of the highest quality and have been thoroughly tested to ensure that they meet all safety standards. We also offer a wide variety of products to suit your specific needs, and our team of experts is always available to assist you with any questions or concerns you may have.  We offer a 20% discount on all orders, and you can use the discount code Jadiyu to receive this discount. Our products are available in a variety of sizes and

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/

